# Load in the Data Using Generator

Each sample in the dataset has a unique ID. In the following code, when defining the generator, we first get the list of ID's from the RGB folder, then use the list of ID's to load the corresponding information of each sample. Associated with each sample is the following information (size and directory name for the training set are also listed):
    1. RGB image (512 * 512 * 3) (train/images/rgb)
    2. NIR image (512 * 512 * 1) (train/images/rgb)
    3. Boundary (512 * 512 * 1) (train/boundaries)
    4. Mask (512 * 512 * 1) (train/masks)
The task of this project is to use the information above to predict for each pixel, which of the six categories it belons to:
    1. Cloud shadow (512 * 512 * 1) (train/labels/cloud_shadow)
    2. Double plant (512 * 512 * 1) (train/labels/double_plant)
    3. Planter skip (512 * 512 * 1) (train/labels/planter_skip)
    4. Standing water (512 * 512 * 1) (train/labels/standing_water)
    5. Waterway (512 * 512 * 1) (train/labels/waterway)
    6. Weed cluster (512 * 512 * 1) (train/labels/weed_cluster)
If a pixel does not belong to any category above, it is considered to be
    0. Background

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import image as mpimg
from matplotlib import pyplot as plt
from typing import Callable, Union
from itertools import product
from scipy import stats

#keras imports
import keras
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.merge import concatenate, add
from keras.layers.convolutional import Conv2DTranspose
from keras.utils import np_utils
from keras.models import Model
from keras.callbacks import Callback
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from tensorflow.keras.utils import to_categorical

%env SM_FRAMEWORK=tf.keras
import albumentations as A
import segmentation_models as sm 
import cv2

In [ ]:
def id_check(path, ntail, ids):
    fignames = os.listdir(path)
    for fig_id in ids:
        if not fig_id + ntail in fignames:
            return False
    return True


def make_label(raw_labels):
    """
    This function creates a 7-channel label from the six labels (add a background channel).
    """
    stacked_labels = np.stack(raw_labels, axis=-1)
    last_label = np.sum(stacked_labels, axis=-1) + 1
    last_label[last_label > 1] = 0
    # Put the background as the last channel of the label
    full_labels =  np.concatenate([stacked_labels, last_label.reshape((512, 512, 1))], axis=-1)
    return full_labels / np.linalg.norm(full_labels, axis=-1).reshape((512, 512, 1))
    
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

def img_gen(dataset='train'):
    rgb_path = os.path.join(dataset, 'images', 'rgb')
    nir_path = os.path.join(dataset, 'images', 'nir')
    bdry_path = os.path.join(dataset, 'boundaries')
    mask_path = os.path.join(dataset, 'masks')
    label_names = ['cloud_shadow', 'double_plant', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']
    label_paths = [os.path.join(dataset, 'labels', label_name) for label_name in label_names]
    label = np.zeros((512,512,7))
    
    rgb_fig_names = os.listdir(rgb_path)
    fig_ids = [fname[:-4] for fname in rgb_fig_names]
    for fig_id in fig_ids:
        rgb_img = mpimg.imread(os.path.join(rgb_path, fig_id + '.jpg'))
        nir_img = mpimg.imread(os.path.join(nir_path, fig_id + '.jpg')).reshape((512, 512, 1))
        bdry_img = mpimg.imread(os.path.join(bdry_path, fig_id + '.png'))
        mask_img = mpimg.imread(os.path.join(mask_path, fig_id + '.png'))
        if dataset != "test":
            label_imgs = [mpimg.imread(os.path.join(label_path, fig_id + '.png')) for label_path in label_paths]
            label = make_label(label_imgs)
            
        input_img = np.concatenate([rgb_img, nir_img], axis=2) / 255. # Concatenate the RGB and NIR
        
        yield fig_id, input_img, bdry_img, mask_img, label
    
   
    
def masked_img_gen(dataset='train', use_augmentation=False):
    aug = A.Compose([  
        A.VerticalFlip(p=0.5),              
        A.RandomRotate90(p=0.5),]) 
    
    for fig_id, input_img, bdry_img, mask_img, label in img_gen(dataset):

        
        final_mask = np.multiply(bdry_img, mask_img).reshape(512, 512, 1) # Form the final mask
        masked_img = np.multiply(final_mask, input_img)
        masked_img = masked_img.reshape(-1, 512, 512, 4)
        
        if use_augmentation:
            augmented = aug(image=masked_img, mask=label)
            masked_img = augmented['image']
            label = augmented['mask']
                    
        if dataset != 'test':
            label = label.reshape(-1, 512, 512, 7)
            yield masked_img, label

        else:
            yield masked_img, fig_id
            
def rgb_masked_img_gen(dataset='train', use_augmentation=False):
    if dataset != 'test':
        for masked_img, label in masked_img_gen(dataset, use_augmentation):
            yield masked_img[:,:,:,0:3], label
        
    else:
        for masked_img, fig_id in masked_img_gen(dataset, use_augmentation):
             yield masked_img[:,:,:,0:3], fig_id

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
               kernel_initializer = 'he_normal', padding = 'same', activation = 'relu')(input_tensor)
    
    x = BatchNormalization()(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same', activation = 'relu')(x)
    
    return x

def createUnet(input_img, n_filters = 16, dropout = 0.1):
    #Down the Unet
    conv1 = conv2d_block(input_img, n_filters)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(dropout)(pool1)
    
    conv2 = conv2d_block(pool1, n_filters*2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(dropout)(pool2)
    
    conv3 = conv2d_block(pool2, n_filters*4)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(dropout)(pool3)
    
    conv4 = conv2d_block(pool3, n_filters*8)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout)(pool4)
    #bottom of the unet
    conv5 = conv2d_block(pool4, n_filters*16)
    
    #up the U
    uconv4 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(conv5)
    uconv4 = concatenate([uconv4, conv4])
    uconv4 = conv2d_block(uconv4, n_filters*8)
    
    uconv3 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(uconv4)
    uconv3 = concatenate([uconv3, conv3])
    uconv3 = conv2d_block(uconv3, n_filters*4)
    
    uconv2 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(uconv3)
    uconv2 = concatenate([uconv2, conv2])
    uconv2 = conv2d_block(uconv2, n_filters*2)
    
    uconv1 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(uconv2)
    uconv1 = concatenate([uconv1, conv1])
    uconv1= conv2d_block(uconv1, n_filters*1)
    
    output = Conv2D(filters = 7, kernel_size = (1,1), activation = "softmax", padding = 'same')(uconv1)
    model = Model(inputs=[input_img], outputs=[output])
    return model

    

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
        
    return loss

def multiclass_weighted_dice_loss(class_weights: Union[list, np.ndarray, tf.Tensor]) -> Callable[[tf.Tensor, tf.Tensor], tf.Tensor]:
    """
    Weighted Dice loss.
    Used as loss function for multi-class image segmentation with one-hot encoded masks.
    :param class_weights: Class weight coefficients (Union[list, np.ndarray, tf.Tensor], len=<N_CLASSES>)
    :return: Weighted Dice loss function (Callable[[tf.Tensor, tf.Tensor], tf.Tensor])
    """
    if not isinstance(class_weights, tf.Tensor):
        class_weights = tf.constant(class_weights)

    def loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        """
        Compute weighted Dice loss.
        :param y_true: True masks (tf.Tensor, shape=(<BATCH_SIZE>, <IMAGE_HEIGHT>, <IMAGE_WIDTH>, <N_CLASSES>))
        :param y_pred: Predicted masks (tf.Tensor, shape=(<BATCH_SIZE>, <IMAGE_HEIGHT>, <IMAGE_WIDTH>, <N_CLASSES>))
        :return: Weighted Dice loss (tf.Tensor, shape=(None,))
        """
        axis_to_reduce = range(1, K.ndim(y_pred))  # Reduce all axis but first (batch)
        numerator = y_true * y_pred * class_weights  # Broadcasting
        numerator = 2. * K.sum(numerator, axis=axis_to_reduce)

        denominator = (y_true + y_pred) * class_weights # Broadcasting
        denominator = K.sum(denominator, axis=axis_to_reduce)

        return 1 - numerator / denominator

    return loss

In [ ]:
learning_rate_reduction=ReduceLROnPlateau(monitor="MeanIoU", patience=2, verbose=0, factor=0.5, min_lr=0.0001)

train_path = os.path.join('train', 'images', 'rgb')
val_path = os.path.join('val', 'images', 'rgb')

steps_per_epoch = len(os.listdir(train_path))//32
validation_steps = len(os.listdir(val_path))//32

print("Steps per epoch: {}".format(steps_per_epoch))
print("Validation steps: {}".format(validation_steps))

In [ ]:
keras.backend.clear_session()
input_img = Input((512, 512, 4), name='img')
model = createUnet(input_img, n_filters = 32, dropout = 0.2)

weights = 5 * np.ones((7,), dtype=np.float32)
weights[-1] = .1
print(weights)

model.compile(optimizer="adam", loss=multiclass_weighted_dice_loss(weights), metrics = [tf.keras.metrics.MeanIoU(num_classes=7, name = "MeanIoU")])
# model.compile(optimizer="adam", loss=weighted_categorical_crossentropy(weights), metrics = [tf.keras.metrics.MeanIoU(num_classes=7, name = "MeanIoU")])


model.summary()

In [ ]:
# Train the model, doing validation at the end of each epoch.
epochs = 30
model.fit(masked_img_gen('train', False), epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=masked_img_gen('val'), validation_steps=validation_steps, use_multiprocessing=False, shuffle=True,\
         callbacks = [learning_rate_reduction])

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
# x_train, y_train, x_val, y_val = load_data(...)

# # preprocess input
# x_train = preprocess_input(x_train)
# x_val = preprocess_input(x_val)

weights = 2 * np.ones((7,))
weights[-1] = 0.5

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet', input_shape = (512,512,3), classes = 7, activation = 'softmax')
model.compile(
    'Adam',
    loss=sm.losses.DiceLoss(class_weights = weights),
    metrics=[sm.metrics.IOUScore(class_weights = weights, name= "MeanIoU")],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
# model.fit(,
#    batch_size=16,
#    epochs=100,
#    validation_data=(x_val, y_val),
# )
epochs = 40
tf.keras.backend.clear_session()
# model.summary()
model.fit(rgb_masked_img_gen('train'), epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=rgb_masked_img_gen('val'), validation_steps=validation_steps, use_multiprocessing=False, shuffle=True)

# Output images

In [ ]:
from PIL import Image
def outputTestImages(model, gen, ensemble):
    if not os.path.isdir('preds'):
        os.mkdir('preds')

    for mask_img, fig_id in gen('test'):
        y_pred = None
        if ensemble:
            y_pred = model(mask_img, 0)
        else:
            y_pred= model.predict(mask_img, verbose=0)

        pred_result = y_pred[0,:,:,:] # take the ith predictio


        # Since we appended the background to the last channel, we need to bring it to the front when saving predictions
        processed_result = np.concatenate([pred_result[:, :, -1].reshape((512, 512, 1)), pred_result[:, :, :-1]], axis=-1)
    #     print(processed_result.shape) # Should be (512, 512, 7)
        # Convert the 7-channel result to 1-channel result and cast to uint8

        final_pred = np.argmax(processed_result, axis=-1).astype(np.uint8)
    #     print(final_pred.shape) # Should be (512, 512)
        # Save the prediction
        filename = os.path.join('preds', fig_id + '.png')
        Image.fromarray(final_pred).save(filename)

# Ensemble model

In [ ]:
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = output = Conv2D(filters = 3, kernel_size = (1,1), activation = "relu", padding = 'same')(merge)
    output = output = Conv2D(filters = 7, kernel_size = (1,1), activation = "softmax", padding = 'same')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
#     plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [ ]:


def buildAndTrainEnsemble(numUnet, lossFunc, optimizer, epochs):
    keras.backend.clear_session()
    models = []
    input_img = Input((512, 512, 4), name='img')
    
    for i in range (numUnet):
        
        
        model = createUnet(input_img)
        
        # weights = np.ones((7,7))
        # weights[:,6] = 1.3
        # weights[6,:] = 1.3
        model.compile(optimizer=optimizer, loss=lossFunc, metrics = [tf.keras.metrics.MeanIoU(num_classes=7)])
        models.append(model)
    for m in models:
        m.fit(masked_img_gen('train'), epochs=epochs, steps_per_epoch=403, validation_data=masked_img_gen('val'), validation_steps=138, use_multiprocessing=False, shuffle=True)
    return models
    
def predictEnsemble(mods):
    def predict(img, verbose):
        preds = None
        for m in mods:
            pred = m.predict(img, verbose = verbose)
#             print (pred[0,0,0])
#             for i in range(7):
#                 pred[:, :, :,i] *= i+1
#             pred = np.sum(pred, axis= 3)
#             print (pred[0,0,0])
            if preds is None:
                preds = pred
            else:
#                 np.stack([preds, pred], axis = -1)
#                 np.dstack((preds, pred))
                preds += pred
#         print(preds.shape)
#         print(to_categorical(preds, 7)[0,0,0])
#         results = stats.mode(preds, axis = 3)
#         preds = preds.reshape((preds.shape[0], preds.shape[1]*preds.shape[2]))
        return preds
    return predict

In [ ]:
adamOpt = keras.optimizers.Adam(lr = .0001)
weights = 10 * np.ones((7,), dtype=np.float32)
weights[0] = .3
lossFunc = multiclass_weighted_dice_loss(weights)

models = buildAndTrainEnsemble(3, lossFunc, adamOpt, 20)

# print(adamOpt.lr)

In [ ]:
new_gen = masked_img_gen('val', False)
new_img_gen = img_gen('val')

In [ ]:
mask_img, label = next(new_gen)
fig_id, input_img, bdry_img, mask_img_old, label_old = next(new_img_gen)
visualize(
    original=input_img[:,:,0:3],
    original_nir = input_img[:,:,3],
    masked_image=mask_img[0,:,:,0:3], 
    masked_nir=mask_img[0,:,:,3],
#     idx_0=label[0,:,:,0],
#     idx_1=label[0,:,:,1],
#     idx_2=label[0,:,:,2],
#     idx_3=label[0,:,:,3],
    idx_4=label[0,:,:,4],
#     idx_5=label[0,:,:,5],
    background=label[0,:,:,6]
)

In [ ]:
print(np.sum(label))

In [ ]:

y_pred= predictEnsemble(models)(mask_img, verbose=0)

pred_result = y_pred[0,:,:,:] # take the ith predictio

processed_result = np.concatenate([pred_result[:, :, -1].reshape((512, 512, 1)), pred_result[:, :, :-1]], axis=-1)

final_pred = np.argmax(processed_result, axis=-1).astype(np.uint8)


In [ ]:

y_pred= model.predict(mask_img, verbose=0)

pred_result = y_pred[0,:,:,:] # take the ith predictio

processed_result = np.concatenate([pred_result[:, :, -1].reshape((512, 512, 1)), pred_result[:, :, :-1]], axis=-1)

print(processed_result.shape)

final_pred = np.argmax(processed_result, axis=-1).astype(np.uint8)

In [ ]:
visualize(bg=processed_result[:,:,0])

In [ ]:
print(np.sum(final_pred))

In [ ]:
final_label = np.argmax(label[0,:,:,:], axis=-1).astype(np.uint8)

In [ ]:
print(label[0,0,0,:])
print(processed_result[0,0,:])
print(np.argmax(processed_result[0,0,:]))
a = tf.one_hot(tf.argmax(processed_result[0,0,:]), depth = 7)
print(a)

In [ ]:
stuff = next(img_gen('val'))

In [ ]:
outputTestImages(predictEnsemble(models), masked_img_gen, True)

In [ ]:
outputTestImages(model, masked_img_gen, False)

In [ ]:
steps_per_epoch = 12901//32
print(steps_per_epoch)

In [ ]:
validation_steps = 4431//32
print(validation_steps)